In [1]:
import os

data from: https://cds.climate.copernicus.eu/cdsapp#!/dataset/insitu-gridded-observations-global-and-regional?tab=doc

https://crudata.uea.ac.uk/cru/data/hrg/index.htm#current

https://crudata.uea.ac.uk/cru/data/hrg/cru_ts_4.07/

https://www.nature.com/articles/s41597-020-0453-3

CRU has a days wet per month variable! that's perfect, even better than precipitation

WET
also has
DTR
which is diurnal temperature range! I need to get that one too

Note 5 in the nature paper confirms it! The min and max temps are means of the daily min max!

In [2]:
from scipy.io import netcdf
import xarray as xr
import numpy as np

In [65]:
import geopandas as gpd

In [68]:
import shapely

In [53]:
class TravelMap:

    def __init__(self, data_dir='/home/shrey/Dev/stats_curriculum/geospatial/data/'):

        self.load_data(data_dir)
        self.process_data()
        self.queried_dfs = []

    def load_data(self, data_dir):
        datafiles = os.listdir(data_dir)
        df_list = []
        for file in datafiles:
            ds = xr.open_dataset(data_dir+file)
            df = ds.to_dataframe().reset_index().dropna()
            df_list.append(df)
        df_list = [df.drop(['stn'], axis=1) for df in df_list] # don't need the 'st' column
        self.df_list = df_list

    def process_data(self):
        df_t= self.df_list[0]
        for df in self.df_list[1:]:
            df_t = df_t.merge(df, on=['lon','lat','time'])

        def cels_to_fahr(x):
            """Americuhhh fuck yeahhh"""
            y = 32 + (x*1.8 )
            return y
        
        df_t['tmx_f'] = df_t['tmx'].apply(lambda x: cels_to_fahr(x))
        df_t['tmn_f'] = df_t['tmn'].apply(lambda x: cels_to_fahr(x))
        df_t['tmp_f'] = df_t['tmp'].apply(lambda x: cels_to_fahr(x))

        self.global_df = df_t    
        #self.global_df.reset_index()
        self.global_df['month'] = self.global_df['time'].dt.month
        self.global_df_monthly = self.global_df.drop(['time'],axis=1).groupby(['lat','lon','month']).mean().reset_index()

    def massage_data(self, data_dir):
        
        datafiles = os.listdir(data_dir)
        df_list = []
        for file in datafiles:
            ds = xr.open_dataset(data_dir+file)
            df = ds.to_dataframe().reset_index().dropna()
            df_list.append(df)
        df_list = [df.drop(['stn'], axis=1) for df in df_list] # don't need the 'st' column
        df_t= df_list[0]
        for df in df_list[1:]:
            df_t = df_t.merge(df, on=['lon','lat','time'])

        def cels_to_fahr(x):
            """Americuhhh fuck yeahhh"""
            y = 32 + (x*1.8 )
            return y
        
        df_t['tmx_f'] = df_t['tmx'].apply(lambda x: cels_to_fahr(x))
        df_t['tmn_f'] = df_t['tmn'].apply(lambda x: cels_to_fahr(x))
        df_t['tmp_f'] = df_t['tmp'].apply(lambda x: cels_to_fahr(x))

        self.global_df = df_t

    def query_by_lat_lon(self, my_lat = 37.77, my_lon = -122.44):
        """
        Queries the global dataframe by lat and lon and then stories the query as well as the results
        """
        #self.queried_df = self.global_df.loc[(np.abs(self.global_df.lon-my_lon)<=0.25) & (np.abs(self.global_df_monthly.lat - my_lat)<=0.25)].groupby('time').agg('mean')

        self.queried_df = self.global_df_monthly.loc[(np.abs(self.global_df_monthly.lon-my_lon)<=0.25) & (np.abs(self.global_df_monthly.lat - my_lat)<=0.25)]
        results_dict = {'lat':my_lat, 'lon':my_lon, 'df':self.queried_df}
        self.queried_dfs.append(results_dict)

In [54]:
my_map = TravelMap()
my_map.query_by_lat_lon()

In [55]:
my_map.global_df_monthly

,lat,lon,month,pre,tmn,tmx,dtr,wet,tmp,tmx_f,tmn_f,tmp_f
0,-55.75,-68.25,1,66.050003,4.750000,12.300000,7.55,12 days 10:11:59.975280760,8.500000,54.140000,40.550000,47.300000
1,-55.75,-68.25,2,53.799999,5.400000,13.150001,7.75,9 days 17:38:23.970336914,9.250000,55.670001,41.720000,48.650000
2,-55.75,-68.25,3,85.850006,4.150000,11.100000,6.95,10 days 00:14:23.978576661,7.600000,51.980000,39.470000,45.680000
3,-55.75,-68.25,4,37.400002,2.800000,9.500000,6.70,8 days 04:11:59.975280760,6.100000,49.100001,37.040000,42.980000
4,-55.75,-68.25,5,40.900002,2.150000,7.400000,5.25,9 days 09:57:35.955505373,4.750000,45.320000,35.870000,40.550000
...,...,...,...,...,...,...,...,...,...,...,...,...
809035,83.75,-28.75,8,5.000000,1.150000,6.600000,5.45,5 days 00:00:00,3.850000,43.880000,34.070000,38.930000
809036,83.75,-28.75,9,7.000000,-9.800000,-5.150000,4.65,7 days 00:00:00,-7.450000,22.730000,14.360000,18.590000
809037,83.75,-28.75,10,0.000000,-23.350000,-17.000000,6.35,0 days 00:00:00,-20.150002,1.399998,-10.030001,-4.270001
809038,83.75,-28.75,11,0.000000,-28.750000,-21.750000,7.00,0 days 00:00:00,-25.250000,-7.150000,-19.750002,-13.450002


In [57]:
my_map.queried_dfs[0]['df']

,lat,lon,month,pre,tmn,tmx,dtr,wet,tmp,tmx_f,tmn_f,tmp_f
382692,37.75,-122.25,1,60.500000,6.850000,16.250000,9.400000,5 days 23:52:47.995262147,11.500000,61.250001,44.330000,52.700000
382693,37.75,-122.25,2,13.750000,6.900000,17.900002,11.000000,3 days 07:12:00.001029969,12.400000,64.220001,44.420000,54.320000
382694,37.75,-122.25,3,32.800003,8.100000,18.400002,10.300000,6 days 23:31:12.001647950,13.250000,65.120001,46.580000,55.850001
382695,37.75,-122.25,4,13.400000,9.500000,20.049999,10.550000,3 days 09:14:23.983726499,14.750000,68.090000,49.100000,58.550001
382696,37.75,-122.25,5,0.250000,11.150000,21.250000,10.100000,0 days 05:02:24.000077245,16.200001,70.250002,52.070000,61.160001
382697,37.75,-122.25,6,3.300000,13.800000,24.200001,10.400000,0 days 20:02:23.998146054,18.950001,75.560001,56.840000,66.110000
382698,37.75,-122.25,7,0.350000,14.350000,24.549999,10.200001,0 days 05:02:24.000077249,19.400002,76.190000,57.830000,66.920001
382699,37.75,-122.25,8,0.650000,14.950001,24.700001,9.750000,0 days 05:09:35.999423862,19.799999,76.460001,58.910001,67.640000
382700,37.75,-122.25,9,3.650000,15.550000,25.600000,10.050000,1 days 09:07:11.999588010,20.549999,78.080001,59.990000,68.990000
382701,37.75,-122.25,10,52.750000,12.250000,21.850000,9.600000,3 days 22:04:47.997499104,17.050001,71.330001,54.050000,62.690001


In [58]:
my_map.query_by_lat_lon(my_lat = 40.7, my_lon = -73.9)

In [61]:
tdf = my_map.queried_dfs[1]['df']

In [63]:
tdf.loc[tdf.wet<10]

TypeError: Invalid comparison between dtype=timedelta64[ns] and int

# next steps

ok I think I want to incorporate the global_df as a dataframe and then write a function that first displays the info--as a function of month though, so first groupby agg by month (DONE). ok now we want to take it in and display it? or we want an operator that takes in any one of these? because there's going to be subsetting logic too right? so we can subset based on params and then redeploy the map to highlight something.

so easiest way to test this would be to cut the map in half and then test the geodeploy method on it

i think something like this will work:
https://stackoverflow.com/questions/73579250/using-python-to-plot-gridded-map

In [69]:

BOXES = 500
a, b, c, d = (-180.0, -90.0, 180.0, 90.0) # global bounds

# create a grid for Califormia, could be a city
gdf_grid = gpd.GeoDataFrame(
    geometry=[
        shapely.geometry.box(minx, miny, maxx, maxy)
        for minx, maxx in zip(np.linspace(a, c, BOXES), np.linspace(a, c, BOXES)[1:])
        for miny, maxy in zip(np.linspace(b, d, BOXES), np.linspace(b, d, BOXES)[1:])
    ],
    crs="epsg:4326",
)

In [70]:
gdf_grid

,geometry
0,"POLYGON ((-179.27856 -90.00000, -179.27856 -89..."
1,"POLYGON ((-179.27856 -89.63928, -179.27856 -89..."
2,"POLYGON ((-179.27856 -89.27856, -179.27856 -88..."
3,"POLYGON ((-179.27856 -88.91784, -179.27856 -88..."
4,"POLYGON ((-179.27856 -88.55711, -179.27856 -88..."
...,...
248996,"POLYGON ((180.00000 88.19639, 180.00000 88.557..."
248997,"POLYGON ((180.00000 88.55711, 180.00000 88.917..."
248998,"POLYGON ((180.00000 88.91784, 180.00000 89.278..."
248999,"POLYGON ((180.00000 89.27856, 180.00000 89.639..."
